In [1]:
import os
os.chdir('..')

%load_ext autoreload
%autoreload 2

In [2]:
import torch

from nltk.tokenize import word_tokenize

from src.main import setup_torch, get_corpus
from src.utils import get_latest_model_file
from src.winograd_schema_challenge import (
    analyse_single_wsc, generate_full_sentences, find_missing_wsc_words_in_corpus_vocab, winograd_test
)

from src.wsc_parser import generate_df, generate_json

In [3]:
still_in_english = [
    "It was a summer afternoon, and the dog was sitting in the middle of the lawn. After a while, it got up and moved to a spot under the tree, because the spot under the tree was hot.",
    "It was a summer afternoon, and the dog was sitting in the middle of the lawn. After a while, it got up and moved to a spot under the tree, because the dog was cooler.",
    "I couldn't put the pot on the shelf because the shelf was too tall.",
    "I couldn't put the pot on the shelf because the pot was too high.",
    "There is a pillar between me and the stage, and I can't see around the stage.",
    "There is a pillar between me and the stage, and I can't see the pillar.",
    "They broadcast an announcement, but a subway came into the station and I couldn't hear the subway.",
    "They broadcast an announcement, but a subway came into the station and I couldn't hear over the announcement.",
]

In [4]:
df = generate_df(' '.join(still_in_english))

In [5]:
with open('data/processed/portuguese_capitalized_words.txt', 'r') as capitalized_words_file:
    capitalized_words = [line.strip() for line in capitalized_words_file.readlines()]

def generate_switched_sentence(row):
    switched = row.schema.lower()
    switched = switched.replace(row.substitution_a.lower(), '<PLACEHOLDER>')
    switched = switched.replace(row.substitution_b.lower(), row.substitution_a.lower())
    switched = switched.replace('<PLACEHOLDER>', row.substitution_b.lower())
    
    switched = '. '.join(map(lambda s: s.strip().capitalize(), switched.split('.')))
    
    words = []
    for word in word_tokenize(switched, language='portuguese'):
        if word.capitalize() in capitalized_words:
            word = word.capitalize()
        if word in ['.', ',', '!', '?', ';']:
            words[-1] += word
        else:
            words += [word]
    switched = ' '.join(words).strip()
    
    return switched

df['switched'] = df.apply(generate_switched_sentence, axis=1)

In [6]:
df['correct_sentence'], df['incorrect_sentence'], df['correct_switched'], df['incorrect_switched'] = \
    zip(*df.apply(generate_full_sentences, axis=1))

In [13]:
df['manually_fixed_correct_sentence'], df['manually_fixed_incorrect_sentence'], \
    df['manually_fixed_correct_switched'], df['manually_fixed_incorrect_switched'] = \
    df['correct_sentence'], df['incorrect_sentence'], df['correct_switched'], df['incorrect_switched']

In [27]:
with open('data/processed/manual_fixes_portuguese.json', 'w') as fp:
    json.dump(manual_fixes_json, fp)

In [23]:
for item in manual_fixes_json:
    if item['question_id'] >= 60:
        item['question_id'] += 1
for item in manual_fixes_json:
    if item['question_id'] >= 61:
        item['question_id'] += 1
for item in manual_fixes_json:
    if item['question_id'] >= 62:
        item['question_id'] += 1
for item in manual_fixes_json:
    if item['question_id'] >= 63:
        item['question_id'] += 1
for item in manual_fixes_json:
    if item['question_id'] >= 72:
        item['question_id'] += 1
for item in manual_fixes_json:
    if item['question_id'] >= 73:
        item['question_id'] += 1
for item in manual_fixes_json:
    if item['question_id'] >= 86:
        item['question_id'] += 1
for item in manual_fixes_json:
    if item['question_id'] >= 87:
        item['question_id'] += 1

In [15]:
df[~df.translated].index

Int64Index([60, 61, 62, 63, 72, 73, 86, 87], dtype='int64')

In [26]:
generate_json(df, switched=True)